In [1]:
import gym
env = gym.make("CartPole-v1")

In [2]:
import json
import random

import ray
try:
    from ray.rllib.agents.agent import get_agent_class
except ImportError:
    from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments, run
from ray.tune.experiment import Experiment
from ray.tune.registry import register_env

from ray.tune.schedulers import PopulationBasedTraining

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
# number of parallel workers
N_CPUS = 4
# number of rollouts per training iteration
N_ROLLOUTS = 1

ray.init(num_cpus=N_CPUS)#, object_store_memory=1000000000)

2020-03-10 11:30:17,974	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2020-03-10_11-30-17_973354_6857/logs.
2020-03-10 11:30:18,088	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:36406 to respond...
2020-03-10 11:30:18,236	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:17150 to respond...
2020-03-10 11:30:18,247	INFO services.py:809 -- Starting Redis shard with 1.65 GB max memory.
2020-03-10 11:30:18,299	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2020-03-10_11-30-17_973354_6857/logs.
2020-03-10 11:30:18,302	WARNING services.py:1330 -- WARNING: The default object store size of 2.47 GB will use more than 50% of the available memory on this node (4.73 GB). Consider setting the object store memory manually to a smaller size to avoid memory contention with other applications.
2020-03-10 11:30:18,304	INFO services.py:1475 -- Starting the Plasma object store with 2.47 GB memory usi

{'node_ip_address': '192.168.2.105',
 'redis_address': '192.168.2.105:36406',
 'object_store_address': '/tmp/ray/session_2020-03-10_11-30-17_973354_6857/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-03-10_11-30-17_973354_6857/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2020-03-10_11-30-17_973354_6857'}

In [4]:
def explore(config):
    # ensure we collect enough timesteps to do sgd
    if config["train_batch_size"] < config["sgd_minibatch_size"] * 2:
        config["train_batch_size"] = config["sgd_minibatch_size"] * 2
    # ensure we run at least one sgd iter
    if config["num_sgd_iter"] < 1:
        config["num_sgd_iter"] = 1
    return config

In [5]:
pbt = PopulationBasedTraining(
        time_attr="time_total_s",
        metric="episode_reward_mean",
        mode="max",
        perturbation_interval=4,
        resample_probability=0.25,
        # Specifies the mutations of these hyperparams
        hyperparam_mutations={
            "lambda": lambda: random.uniform(0.9, 1.0),
            "vf_clip_param": lambda: random.uniform(20000, 50000),
            "lr": [5e-2, 1e-3, 5e-4, 1e-4, 5e-5, 1e-5],
            "sgd_minibatch_size": lambda: random.randint(128, 16384),
            "train_batch_size": lambda: random.randint(N_CPUS*HORIZON, 2*N_CPUS*HORIZON),
        },
        custom_explore_fn=explore)

In [6]:
# The algorithm or model to train. This may refer to "
#      "the name of a built-on algorithm (e.g. RLLib's DQN "
#      "or PPO), or a user-defined trainable function or "
# #      "class registered in the tune registry.")
alg_run = "DQN"

# HORIZON = 200
BATCH_SIZE = 200

agent_cls = get_agent_class(alg_run)
config = agent_cls._default_config.copy()
config["num_workers"] = N_CPUS - 1  # number of parallel workers
config["num_envs_per_worker"] = 1  # number of parallel workers
# config["num_gpus"] = 0.1
config["train_batch_size"] = 1000  # batch size
config["sample_batch_size"] = 200  # batch size
config["gamma"] = 0.99  # discount rate
config["model"].update({"fcnet_hiddens": [256]})  # size of hidden layers in network
config["log_level"] = "DEBUG"
config["n_step"] = 2
# config["noisy"] = True
config["num_atoms"] = 2

# save the flow params for replay
# flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True,
#                        indent=4)  # generating a string version of flow_params
# config['env_config']['flow_params'] = flow_json  # adding the flow_params to config dict
# config['env_config']['run'] = alg_run

# Call the utility function make_create_env to be able to 
# register the Flow env for this experiment
# create_env, gym_name = make_create_env(params=flow_params, version=0)

config["env"] = "CartPole-v1"
# Register as rllib env with Gym
# register_env(gym_name, create_env)

In [7]:
exp = Experiment("cart_pole_tests", **{
        "run": alg_run,
        "config": {
            **config
        },
        "checkpoint_freq": 5,  # number of iterations between checkpoints
        "checkpoint_at_end": True,  # generate a checkpoint at the end
        "max_failures": 5,
        "stop": {  # stopping conditions
            "episode_reward_mean": 200,  # number of iterations to stop after
        },
        "num_samples": 1})

In [8]:
trials = run_experiments(exp)

2020-03-10 11:30:18,572	INFO trial_runner.py:176 -- Starting a new experiment.
2020-03-10 11:30:18,628	WARNING signature.py:108 -- The function with_updates has a **kwargs argument, which is currently not supported.
2020-03-10 11:30:18,641	WARNING logger.py:227 -- Could not instantiate <class 'ray.tune.logger.TFLogger'> - skipping.
2020-03-10 11:30:18,643	ERROR log_sync.py:34 -- Log sync requires cluster to be setup with `ray up`.
2020-03-10 11:30:18,750	WARNING util.py:145 -- The `start_trial` operation took 0.1493391990661621 seconds to complete, which may be a performance bottleneck.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/4 CPUs, 0/0 GPUs
Memory usage on this node: 3.6/8.2 GB

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 4/4 CPUs, 0/0 GPUs
Memory usage on this node: 3.6/8.2 GB
Result logdir: /home/valentin/ray_results/cart_pole_tests
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - DQN_CartPole-v1_0:	RUNNING

(pid=6896) WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/compat/v2_compat.py:65: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=6896) Instructions for updating:
(pid=6896) non-resource variables are not supported in the long term
(pid=6896) 2020-03-10 11:30:21,251	INFO rollout_worker.py:319 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
(pid=6896) 2020-03-10 11:30:21,251	DEBUG worker_set.py:135 -- Creating TF session {'intra_op_parallelism_threa

(pid=6896) 2020-03-10 11:30:22,264	WARNING util.py:47 -- Install gputil for GPU system monitoring.
(pid=6896) WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/ray/rllib/policy/tf_policy.py:570: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
(pid=6896) Instructions for updating:
(pid=6896) Prefer Variable.assign which has equivalent behavior in 2.X.
(pid=6896) 2020-03-10 11:30:22,328	DEBUG trainer.py:353 -- updated global vars: {'timestep': 0}
(pid=6897) WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/compat/v2_compat.py:65: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=6897) Instructions for updating:
(pid=6897) non-resource variables are not supported in the long term
(pid=6897) 2020-03-10 11:30:24,519	INFO rollout_worker.py:319 -- Creating policy evaluation worker 2 on CPU (please ignore a

(pid=6897) 2020-03-10 11:30:25,017	DEBUG catalog.py:434 -- Created model <ray.rllib.models.tf.fcnet_v1.FullyConnectedNetwork object at 0x7f1bb4622470>: ({'obs': <tf.Tensor 'default_policy/new_obs:0' shape=(?, 4) dtype=float32>, 'is_training': <tf.Tensor 'default_policy/PlaceholderWithDefault:0' shape=() dtype=bool>} of Box(4,), Discrete(2), [], None) -> Tensor("default_policy/q_func_4/fc_net/fc_out/Tanh:0", shape=(?, 256), dtype=float32), []
(pid=6897) WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/ray/rllib/agents/dqn/dqn_policy.py:64: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
(pid=6897) Instructions for updating:
(pid=6897) Use `tf.cast` instead.
(pid=6897) WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/ray/rllib/agents/dqn/dqn_policy.py:83: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
(pid=6897) Instructions for u

(pid=6897) 2020-03-10 11:30:25,919	DEBUG simple_q_policy.py:60 -- Update target op <tf.Variable 'default_policy/target_q_func/action_value/hidden_0/kernel:0' shape=(256, 256) dtype=float32_ref>
(pid=6897) 2020-03-10 11:30:25,920	DEBUG simple_q_policy.py:60 -- Update target op <tf.Variable 'default_policy/target_q_func/action_value/hidden_0/bias:0' shape=(256,) dtype=float32_ref>
(pid=6897) 2020-03-10 11:30:25,921	DEBUG simple_q_policy.py:60 -- Update target op <tf.Variable 'default_policy/target_q_func/action_value/dense/kernel:0' shape=(256, 4) dtype=float32_ref>
(pid=6897) 2020-03-10 11:30:25,922	DEBUG simple_q_policy.py:60 -- Update target op <tf.Variable 'default_policy/target_q_func/action_value/dense/bias:0' shape=(4,) dtype=float32_ref>
(pid=6897) 2020-03-10 11:30:25,923	DEBUG simple_q_policy.py:60 -- Update target op <tf.Variable 'default_policy/target_q_func/state_value/dense/kernel:0' shape=(256, 256) dtype=float32_ref>
(pid=6897) 2020-03-10 11:30:25,924	DEBUG simple_q_policy

...

Result for DQN_CartPole-v1_0:
  custom_metrics: {}
  date: 2020-03-10_11-31-34
  done: false
  episode_len_mean: 33.19
  episode_reward_max: 153.0
  episode_reward_mean: 33.19
  episode_reward_min: 19.0
  episodes_this_iter: 71
  episodes_total: 4414
  experiment_id: b16aa22f956249e0baca3de8c0962829
  hostname: valentin-Aspire-V3-372
  info:
    grad_time_ms: 80.372
    learner:
      default_policy:
        cur_lr: 0.0005000000237487257
        mean_td_error: 0.39868950843811035
        model: {}
    max_exploration: 0.020000000000000018
    min_exploration: 0.020000000000000018
    num_steps_sampled: 85800
    num_steps_trained: 141000
    num_target_updates: 143
    opt_peak_throughput: 12442.148
    opt_samples: 1000.0
    replay_time_ms: 80.67
    sample_time_ms: 299.723
    update_time_ms: 5.042
  iterations_since_restore: 53
  node_ip: 192.168.2.105
  num_healthy_workers: 3
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 60.849999999999994
    ram_util_percent: 57.5
  p

(pid=6896) 2020-03-10 11:31:52,291	DEBUG trainer.py:353 -- updated global vars: {'timestep': 109200}
(pid=6896) 2020-03-10 11:31:53,344	DEBUG trainer.py:353 -- updated global vars: {'timestep': 110400}
(pid=6896) 2020-03-10 11:31:54,853	DEBUG trainer.py:353 -- updated global vars: {'timestep': 112200}
Result for DQN_CartPole-v1_0:
  custom_metrics: {}
  date: 2020-03-10_11-31-56
  done: false
  episode_len_mean: 108.38
  episode_reward_max: 142.0
  episode_reward_mean: 108.38
  episode_reward_min: 61.0
  episodes_this_iter: 18
  episodes_total: 4641
  experiment_id: b16aa22f956249e0baca3de8c0962829
  hostname: valentin-Aspire-V3-372
  info:
    grad_time_ms: 82.292
    learner:
      default_policy:
        cur_lr: 0.0005000000237487257
        mean_td_error: 0.2639786899089813
        model: {}
    max_exploration: 0.020000000000000018
    min_exploration: 0.020000000000000018
    num_steps_sampled: 114000
    num_steps_trained: 188000
    num_target_updates: 190
    opt_peak_throughp

(pid=6896) 2020-03-10 11:32:13,280	DEBUG trainer.py:353 -- updated global vars: {'timestep': 136800}
(pid=6896) 2020-03-10 11:32:14,331	DEBUG trainer.py:353 -- updated global vars: {'timestep': 138000}
(pid=6896) 2020-03-10 11:32:15,742	DEBUG trainer.py:353 -- updated global vars: {'timestep': 139800}
Result for DQN_CartPole-v1_0:
  custom_metrics: {}
  date: 2020-03-10_11-32-17
  done: false
  episode_len_mean: 131.86
  episode_reward_max: 170.0
  episode_reward_mean: 131.86
  episode_reward_min: 98.0
  episodes_this_iter: 12
  episodes_total: 4878
  experiment_id: b16aa22f956249e0baca3de8c0962829
  hostname: valentin-Aspire-V3-372
  info:
    grad_time_ms: 79.799
    learner:
      default_policy:
        cur_lr: 0.0005000000237487257
        mean_td_error: 0.22711428999900818
        model: {}
    max_exploration: 0.020000000000000018
    min_exploration: 0.020000000000000018
    num_steps_sampled: 141600
    num_steps_trained: 234000
    num_target_updates: 236
    opt_peak_through

(pid=6895) 2020-03-10 11:32:27,498	INFO rollout_worker.py:485 -- Completed sample batch:
(pid=6895) 
(pid=6895) { 'data': { 'actions': np.ndarray((200,), dtype=int64, min=0.0, max=1.0, mean=0.49),
(pid=6895)             'agent_index': np.ndarray((200,), dtype=int64, min=0.0, max=0.0, mean=0.0),
(pid=6895)             'dones': np.ndarray((200,), dtype=bool, min=0.0, max=1.0, mean=0.01),
(pid=6895)             'eps_id': np.ndarray((200,), dtype=int64, min=285505046.0, max=1323685361.0, mean=830549711.375),
(pid=6895)             'infos': np.ndarray((200,), dtype=object, head={}),
(pid=6895)             'new_obs': np.ndarray((200, 4), dtype=float32, min=-2.409, max=0.629, mean=-0.37),
(pid=6895)             'obs': np.ndarray((200, 4), dtype=float32, min=-2.386, max=0.629, mean=-0.362),
(pid=6895)             'prev_actions': np.ndarray((200,), dtype=int64, min=0.0, max=1.0, mean=0.485),
(pid=6895)             'prev_rewards': np.ndarray((200,), dtype=float32, min=0.0, max=1.0, mean=0.995),


Result for DQN_CartPole-v1_0:
  custom_metrics: {}
  date: 2020-03-10_11-32-34
  done: true
  episode_len_mean: 205.38
  episode_reward_max: 316.0
  episode_reward_mean: 205.38
  episode_reward_min: 150.0
  episodes_this_iter: 6
  episodes_total: 4986
  experiment_id: b16aa22f956249e0baca3de8c0962829
  hostname: valentin-Aspire-V3-372
  info:
    grad_time_ms: 89.154
    learner:
      default_policy:
        cur_lr: 0.0005000000237487257
        mean_td_error: 0.16909317672252655
        model: {}
    max_exploration: 0.020000000000000018
    min_exploration: 0.020000000000000018
    num_steps_sampled: 163800
    num_steps_trained: 271000
    num_target_updates: 273
    opt_peak_throughput: 11216.57
    opt_samples: 1000.0
    replay_time_ms: 89.004
    sample_time_ms: 310.092
    update_time_ms: 5.33
  iterations_since_restore: 99
  node_ip: 192.168.2.105
  num_healthy_workers: 3
  off_policy_estimator: {}
  perf:
    cpu_util_percent: 65.15
    ram_util_percent: 59.7
  pid: 6896
  p